## João Pedro Rodrigues Freitas - 11316552

In [25]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

# TODO

- Fazer os mini batches (train, test)
- Fazer o one-hot
- Fazer normalização

# Data

In [26]:
class Data():
    def __init__(self, X: np.ndarray) -> None:
        self.data = X

    def getData(self) -> np.ndarray:
        return self.data

    def setTrainRatio(self, ratio: float) -> None:
        self.train_ratio = ratio

    def getTrainRatio(self) -> float:
        return self.train_ratio
    
    def getAttrSize(self) -> int:
        return self.data.shape[1]
    
    def shuffleData(self) -> None:
        self.data = np.random.permutation(self.data)

    def getTrainData(self) -> np.ndarray:
        return self.data[:int(self.data.shape[0] * self.train_ratio)]
    
    def getTestData(self) -> np.ndarray:
        return self.data[int(self.data.shape[0] * self.train_ratio):]

    def normalize(self) -> None:
        '''Normaliza cada atributo para o intervalo [0, 1]'''
        for i in range(self.getAttrSize()):
            self.data[:,i] = (self.data[:,i] - np.min(self.data[:,i])) / (np.max(self.data[:,i]) - np.min(self.data[:,i]))
    
    # def plotData(self) -> None:
    #     if self.getAttrSize() != 2:
    #         raise ValueError("The number of attributes must be 2")
        
    #     plt.scatter(self.data[:,0], self.data[:,1], c=self.data[:,-1])
    

## Autoencoder

In [27]:
class Autoencoder():

    def __init__(self, layers: int, neuronsByLayer: list, data: Data) -> None:
        if len(neuronsByLayer) != layers:
            raise ValueError("The size of the list must be equal to the number of layers")

        # [input, hidden1, hidden2, ..., latentSpace]
        self.layers = layers
        self.neurons = neuronsByLayer
        self.data = data

        # camada inicial
        self.W = [np.random.rand(data.getAttrSize(), self.neurons[0]) - 0.5]
        self.b = [np.random.rand(1, self.neurons[0]) - 0.5]

        # camadas ocultas
        for i in range(1, self.layers):
            self.W.append(np.random.rand(self.neurons[i-1], self.neurons[i]) - 0.5)
            self.b.append(np.random.rand(1, self.neurons[i]) - 0.5)

        # TODO: parte do decoder

    def getLayers(self) -> int:
        return self.layers

    def getNeuronsByLayer(self) -> list[int]:
        return self.neurons
    
    def getNeuronsOfLayer(self, layer: int) -> int:
        return self.neurons[layer]
    
    def sigmoid(self, Z: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-Z))
    
    def der_sigmoid(self, Z: np.ndarray) -> np.ndarray:
        f = self.sigmoid(Z)
        return f * (1 - f)
    
    def getWeights(self) -> list[np.ndarray]:
        return self.W
    
    def getBiases(self) -> list[np.ndarray]:
        return self.b
    
    def getZ(self) -> list[np.ndarray]:
        return self.Z
    
    def getA(self) -> list[np.ndarray]:
        return self.A
    
    def getdZ(self) -> list[np.ndarray]:
        return self.dZ
    
    def getdW(self) -> list[np.ndarray]:
        return self.dW
    
    def getdb(self) -> list[np.ndarray]:
        return self.db
    
    def forward(self, X: np.ndarray) -> None:
        self.Z = [X] # Z[0] é o input
        self.A = [X] # A[0] é o input

        for i in range(self.layers):
            Z = np.dot(self.A[i], self.W[i]) + self.b[i]

            if i == self.layers - 1:
                A = self.softmax(Z.T)
            else:
                A = self.sigmoid(Z)

            self.Z.append(Z)
            self.A.append(A)

    def backward(self, Y: np.ndarray) -> None:
        # TODO: conferir esse dZ se é 2*(A - Y)
        n = Y.shape[0]

        self.dZ = [2 * (self.A[-1] - Y)] # erro da camada de saída
        self.dW = [np.dot(self.dZ[-1], self.A[-2]) / n]
        self.db = [np.sum(self.dZ[-1], axis=1, keepdims=True) / n]

        for i in range(self.layers-1, 0, -1):
            print("Layer: ", i)
            self.dZ.append(np.dot(self.W[i], self.dZ[-1]) * self.der_sigmoid(self.Z[i].T))
            self.dW.append(np.dot(self.dZ[-1], self.A[i-1]))
            self.db.append(np.sum(self.dZ[-1], axis=1, keepdims=True))

    def adjustParams(self, lr: float) -> None:
        reverseDw = self.dW[::-1]
        reverseDb = self.db[::-1]

        for i in range(self.layers):
            self.W[i] -= lr * reverseDw[i].T
            self.b[i] -= lr * reverseDb[i].T

    def get_preds(self, y):
        return np.argmax(y, axis=0)
    
    def getAccuracy(self, y_hat, y):
        return np.sum(y_hat == y) / y.size
    
    def fit(self, lr: float = 0.01, n_epochs: int = 100) -> None:
        for epoch in range(n_epochs):
            self.forward(self.data.getTrainData()[:,:-1])
            self.backward(self.data.getTrainData()[:,-1])
            self.adjustParams(lr)

            if epoch % 10 == 0:
                print(f"Epoch: {epoch}")
                y_hat = self.get_preds(self.A[-1])
                y_pred = self.data.getTrainData()[:,-1]
                print(f"Loss: {np.mean(np.square(self.data.getTrainData()[:,-1] - self.A[-1]))}")
                print(f"Accuracy: {self.getAccuracy(y_hat, y_pred)}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        self.forward(X)
        return self.A[-1]



# NOVO

In [28]:
class Autoencoder():

    def __init__(self, inputDim: int, hiddenLayers: int, neuronsPerLayer: list[int]) -> None:
        if len(neuronsPerLayer) != hiddenLayers:
            raise ValueError("The size of the list must be equal to the number of hidden layers")

        self.inputDim = inputDim
        self.hiddenLayers = hiddenLayers
        self.neuronsPerLayer = neuronsPerLayer
        self.W = []
        self.b = []
        self._initParams()

    def _initParams(self) -> None:
        # layersDim = [input, hidden1, hidden2 (latentSpace), hidden1, output]
        layersDim = [self.inputDim] + self.neuronsPerLayer + self.neuronsPerLayer[::-1][1:] + [self.inputDim]

        for i in range(1, len(layersDim)):
            # inicializa com distribuição uniforme entre -0.5 e 0.5
            self.W.append(np.random.rand(layersDim[i-1], layersDim[i]) - 0.5)
            self.b.append(np.random.rand(1, layersDim[i]) - 0.5)

        print(self.W)
    
    def _sigmoid(self, x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))
    
    def _der_sigmoid(self, x: np.ndarray) -> np.ndarray:
        return x * (1 - x)
    
    def encode(self, x: np.ndarray) -> np.ndarray:
        a = x
        for i in range(self.hiddenLayers + 1):
            z = np.dot(a, self.W[i]) + self.b[i]
            a = self._sigmoid(z)
        
        return a
    
    def decode(self, x: np.ndarray) -> np.ndarray:
        a = x

        for i in range(self.hiddenLayers + 1, 2 * self.hiddenLayers + 1):
            print(i)
            z = np.dot(a, self.W[i]) + self.b[i]
            a = self._sigmoid(z)
        
        return a
    
    def fit(self, x, lr: float = 0.01, n_epochs: int = 100):
        for epoch in range(n_epochs):

            # Forward
            encoded = self.encode(x)
            decoded = self.decode(encoded)

            # 
            # loss = np.mean(np.square(decoded - x))
            # err = 2 * (decoded - x)

            # # Backward
            




            # # Backward
            # error = decoded - x
            # delta = error * self._der_sigmoid(decoded)
            # deltas = [delta]
            # for i in range(self.hiddenLayers, 0, -1):
            #     delta = np.dot(deltas[-1], self.W[i].T) * self._der_sigmoid(encoded)
            #     deltas.append(delta)

            # deltas = deltas[::-1]

            # # Update weights
            # for i in range(self.hiddenLayers + 1):
            #     self.W[i] -= lr * np.dot(x.T, deltas[i])
            #     self.b[i] -= lr * np.sum(deltas[i], axis=0, keepdims=True)

            # for i in range(self.hiddenLayers + 1, 2 * self.hiddenLayers + 1):
            #     self.W[i] -= lr * np.dot(encoded.T, deltas[i])
            #     self.b[i] -= lr * np.sum(deltas[i], axis=0, keepdims=True)

            # print(f"Epoch: {epoch} - Loss: {loss}")
            # print(self.W)
            # print(self.b)
            # print("")
            
    
    # def fit(self, x, lr=0.01, n_epochs=100):
    #     for epoch in range(n_epochs):
    #         a = x
    #         a_list = [a]
    #         z_list = [a]
    #         for i in range(self.hiddenLayers + 1):
    #             z = np.dot(a, self.W[i]) + self.b[i]
    #             a = self._sigmoid(z)
    #             a_list.append(a)
    #             z_list.append(z)
            
    #         for i in range(self.hiddenLayers + 1, 2 * self.hiddenLayers + 1):
    #             z = np.dot(a, self.W[i]) + self.b[i]
    #             a = self._sigmoid(z)
    #             a_list.append(a)
    #             z_list.append(z)
            
    #         # calcula o erro
    #         loss = np.mean(np.square(x - a))
    #         print(f"Epoch: {epoch} - Loss: {loss}")
            
    #         # calcula os deltas
    #         delta = 2 * (a - x) * self._der_sigmoid(a_list[-1])
    #         deltas = [delta]
    #         for i in range(self.hiddenLayers, 0, -1):
    #             delta = np.dot(deltas[-1], self.W[i].T) * self._der_sigmoid(a_list[i])
    #             deltas.append(delta)
            
    #         deltas = deltas[::-1]
            
    #         # atualiza os pesos
    #         for i in range(self.hiddenLayers + 1):
    #             self.W[i] -= lr * np.dot(a_list[i].T, deltas[i])
    #             self.b[i] -= lr * np.sum(deltas[i], axis=0, keepdims=True)
            
    #         for i in range(self.hiddenLayers + 1, 2 * self.hiddenLayers + 1):
    #             self.W[i] -= lr * np.dot(a_list[i].T, deltas[i])
    #             self.b[i] -= lr * np.sum(deltas[i], axis=0, keepdims=True)
            
    #         print(self.W)
    #         print(self.b)
    #         print("")



In [29]:
def main() -> None:
    X = np.vstack([np.random.normal(1, 0.5, [100,10]),
                   np.random.normal(-2, 1, [100,10]),
                   np.random.normal(3, 0.75, [100,10])])

    data = Data(X)
    
    data.normalize()
    data.shuffleData()
    data.setTrainRatio(0.8)

    inputDim = data.getAttrSize()
    hiddenLayers = 2
    neuronsPerLayer = [5, 3] # Dimensão latente é a ultima camada oculta
    # [10, 5, 3, 5, 10]
    
    autoencoder = Autoencoder(inputDim, hiddenLayers, neuronsPerLayer)
    autoencoder.fit(data.getTrainData(), lr = 0.01, n_epochs = 100)

    # ae = Autoencoder(2, [10, 3], data) # len([input, hidden1, hidden2, ..., latentSpace])

    # ae.forward(data.getTrainData()[:,:-1])
    # ae.backward(data.getTrainData()[:,-1])
    # ae.adjustParams(0.05)


    # print(ae.getWeights()[0].shape) # w1
    # print(ae.getBiases()[0].shape) # b1
    # print(ae.getWeights()[1].shape) # w2
    # print(ae.getBiases()[1].shape) # b2
    # print(ae.getZ()[1].shape) # O1
    # print(ae.getA()[1].shape) # A1
    # print(ae.getZ()[2].shape) # O2
    # print(ae.getA()[2].shape) # A2


    # print(ae.getdW()[1].shape) # dW1
    # print(ae.getdb()[1].shape) # db1
    # print(ae.getdW()[0].shape) # dW2
    # print(ae.getdb()[0].shape) # db2




    # ae.fit(lr = 0.05, n_epochs = 200)

    # y_hat = ae.predict(data.getTestData()[:,:-1])
    # print(ae.getAccuracy(y_hat, ae.get_preds(data.getTestData()[:,-1])))
    

In [30]:
if __name__ == "__main__":
    main()

[array([[-0.15459429,  0.0899398 , -0.37538827,  0.21014735,  0.43031858],
       [-0.19920491,  0.30950532,  0.4749302 , -0.18516199, -0.34896075],
       [ 0.46485666,  0.1242544 , -0.33730847,  0.08141972, -0.07104045],
       [-0.43659876,  0.39311029,  0.3354205 , -0.29398829,  0.33445697],
       [ 0.28824607,  0.24147265, -0.26062148, -0.46254485,  0.31806867],
       [ 0.28704131, -0.30544349, -0.23599799, -0.2560688 ,  0.02361333],
       [ 0.22764095, -0.221249  , -0.31868874,  0.21811137,  0.43768673],
       [-0.13571291, -0.01671031,  0.41388737,  0.27256543, -0.32503961],
       [-0.2948496 , -0.44640027, -0.22974705, -0.0266718 , -0.19580032],
       [-0.02864194,  0.11603893,  0.43087347,  0.42349679,  0.12226805]]), array([[-0.44139401, -0.14647768,  0.45454106],
       [-0.27545203, -0.18978752, -0.20715384],
       [ 0.20136612,  0.23501206, -0.05568166],
       [ 0.4128185 ,  0.22524653,  0.38797923],
       [ 0.35209704,  0.27259301, -0.17906927]]), array([[ 0.2529

IndexError: list index out of range